# Import Libraries

In [1]:
import pandas as pd
from os import path
import sqlite3

# Load Data

In [2]:
# handle directories
DATA_DIR = r'C:\Users\Maintenant pret\OneDrive\Documents\Learn to Code With Basketball\code-basketball-files-main\data'

# create connection
conn = sqlite3.connect(path.join(DATA_DIR, 'basketball-data.sqlite'))

# load csv data
player_game = pd.read_csv(path.join(DATA_DIR, 'player_game.csv'))
player = pd.read_csv(path.join(DATA_DIR, 'players.csv'))
game = pd.read_csv(path.join(DATA_DIR, 'games.csv'))
team = pd.read_csv(path.join(DATA_DIR, 'teams.csv'))

# Write it to SQL

In [3]:
# write it to sql
player_game.to_sql('player_game', conn, index=False, if_exists='replace')
player.to_sql('player', conn, index=False, if_exists='replace')
game.to_sql('game', conn, index=False, if_exists='replace')
team.to_sql('team', conn, index=False, if_exists='replace')

30

# Read SQL

In [4]:
df = pd.read_sql(
    """
    SELECT *
    FROM player
    """, conn)

df.head()

,player_id,first,last,name,birthdate,school,country,last_affiliation,height,weight,...,rosterstatus,from_year,dleague_flag,draft_year,draft_round,draft_number,team_id,team_id2,team,team2
0,1713,Vince,Carter,V. Carter,19770126,North Carolina,USA,North Carolina/USA,6-6,220.0,...,Inactive,1998.0,N,1998,1.0,5.0,1610612737,0,ATL,None
1,2037,Jamal,Crawford,J. Crawford,19800320,Michigan,USA,Michigan/USA,6-5,200.0,...,Inactive,2000.0,N,2000,1.0,8.0,1610612751,0,BKN,None
2,2544,LeBron,James,L. James,19841230,St. Vincent-St. Mary HS (OH),USA,St. Vincent-St. Mary HS (OH)/USA,6-9,250.0,...,Active,2003.0,N,2003,1.0,1.0,1610612747,0,LAL,None
3,2546,Carmelo,Anthony,C. Anthony,19840529,Syracuse,USA,Syracuse/USA,6-7,238.0,...,Active,2003.0,N,2003,1.0,3.0,1610612757,0,POR,None
4,2594,Kyle,Korver,K. Korver,19810317,Creighton,USA,Creighton/USA,6-7,212.0,...,Inactive,2003.0,N,2003,2.0,51.0,1610612749,0,MIL,None


In [5]:
df = pd.read_sql(
    """
    SELECT player_id, name, birthdate as bday
    FROM player
    WHERE school NOT IN ('North Carolina', 'Duke')
    """, conn)

df.head()

,player_id,name,bday
0,2037,J. Crawford,19800320
1,2544,L. James,19841230
2,2546,C. Anthony,19840529
3,2594,K. Korver,19810317
4,2730,D. Howard,19851208


In [6]:
df = pd.read_sql(
    """
    SELECT player_id, name, birthdate as bday
    FROM player
    WHERE country = 'Canada'
    """, conn)

df

,player_id,name,bday
0,202684,T. Thompson,19910313
1,202709,C. Joseph,19910820
2,203482,K. Olynyk,19910419
3,203920,K. Birch,19920928
4,203939,D. Powell,19910720
5,203952,A. Wiggins,19950223
6,1626168,T. Lyles,19951105
7,1627750,J. Murray,19970223
8,1628415,D. Brooks,19960122
9,1628513,N. Mitrou-Long,19930803


# Join or Select from multiple tables

In [7]:
df = pd.read_sql(
    """
    SELECT 
    player.name,
    player.pos,
    player.team,
    team.conference,
    team.division,
    player_game.*
    FROM player, team, player_game
    WHERE player.team = team.team AND
    player_game.player_id = player.player_id
    """, conn)

df.head()

,name,pos,team,conference,division,name,fgm,fga,fg_pct,pts,...,min,season,player_id,pos,team,date,game_id,plus_minus,dd2,start
0,V. Carter,Guard,ATL,East,Southeast,V. Carter,1,1,1.000,2,...,3.950000,2019-20,1713,Guard,ATL,20200226,21900867,-8,0,0
1,V. Carter,Guard,ATL,East,Southeast,V. Carter,2,5,0.400,5,...,12.658333,2019-20,1713,Guard,ATL,20200311,21900969,6,0,0
2,V. Carter,Guard,ATL,East,Southeast,V. Carter,2,5,0.400,6,...,11.883333,2019-20,1713,Guard,ATL,20191202,21900295,12,0,0
3,J. Crawford,Guard,BKN,East,Atlantic,J. Crawford,2,4,0.500,5,...,5.966667,2019-20,2037,Guard,BKN,20200804,21901256,4,0,0
4,L. James,Forward,LAL,West,Pacific,L. James,6,19,0.316,16,...,34.300000,2019-20,2544,Forward,LAL,20200730,21901232,-4,1,1


In [8]:
df.shape

(2114, 36)

In [9]:
df = pd.read_sql(
    """
    SELECT *
    FROM player
    LIMIT 5
    """, conn)

df

,player_id,first,last,name,birthdate,school,country,last_affiliation,height,weight,...,rosterstatus,from_year,dleague_flag,draft_year,draft_round,draft_number,team_id,team_id2,team,team2
0,1713,Vince,Carter,V. Carter,19770126,North Carolina,USA,North Carolina/USA,6-6,220.0,...,Inactive,1998.0,N,1998,1.0,5.0,1610612737,0,ATL,None
1,2037,Jamal,Crawford,J. Crawford,19800320,Michigan,USA,Michigan/USA,6-5,200.0,...,Inactive,2000.0,N,2000,1.0,8.0,1610612751,0,BKN,None
2,2544,LeBron,James,L. James,19841230,St. Vincent-St. Mary HS (OH),USA,St. Vincent-St. Mary HS (OH)/USA,6-9,250.0,...,Active,2003.0,N,2003,1.0,1.0,1610612747,0,LAL,None
3,2546,Carmelo,Anthony,C. Anthony,19840529,Syracuse,USA,Syracuse/USA,6-7,238.0,...,Active,2003.0,N,2003,1.0,3.0,1610612757,0,POR,None
4,2594,Kyle,Korver,K. Korver,19810317,Creighton,USA,Creighton/USA,6-7,212.0,...,Inactive,2003.0,N,2003,2.0,51.0,1610612749,0,MIL,None


In [10]:
df = pd.read_sql(
    """
    SELECT season, date
    FROM game
    """, conn)

df.head()

,season,date
0,2019-20,2019-10-22
1,2019-20,2019-10-22
2,2019-20,2019-10-23
3,2019-20,2019-10-23
4,2019-20,2019-10-23


# End of Chapter Exercises

In [11]:
df = pd.read_sql(
    """
    SELECT
        date, name, fgm, fga, pts
    FROM player_game, team
    WHERE
        team.team = player_game.team AND
        team.division = 'Central'
    """, conn)

df.head()

,date,name,fgm,fga,pts
0,20200229,A. Drummond,12,21,27
1,20191207,A. McKinnie,2,7,5
2,20191118,A. McKinnie,4,10,14
3,20191207,A. Zizic,2,3,5
4,20200107,A. Zizic,2,3,5


In [12]:
df = pd.read_sql(
    """
    SELECT
        p.first, p.last, date, fgm, fga, pts
    FROM player_game AS pg, team AS t, player AS p
    WHERE
        t.team = pg.team AND
        t.division = 'Central' AND p.player_id = pg.player_id
    """, conn)

df.head()

,first,last,date,fgm,fga,pts
0,Dean,Wade,20191118,0,1,0
1,Dean,Wade,20200107,1,1,3
2,Cedi,Osman,20191207,1,2,2
3,Matthew,Dellavedova,20191118,1,2,5
4,Matthew,Dellavedova,20200107,1,3,3


In [13]:
df.shape

(258, 6)